In [3]:
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import  Chroma
from langchain.document_loaders import JSONLoader
import chromadb

In [4]:

model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-29 01:16:25.379207: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computa

In [18]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["testament"] = record.get("testament")
    metadata["book"] = record.get("book")
    metadata["chapter"] = record.get("chapter")
    metadata["verse"] = record.get("verse")
    metadata["title"] = f"""{record.get("book")} {record.get("chapter")}:{record.get("verse")}"""
    return metadata

In [21]:
dir_path="./data/bible_data_v2.fr.json"
chroma_instance = chromadb.Client()
doc_loader = JSONLoader(dir_path,".[]",content_key="text", metadata_func=metadata_func)
docs = doc_loader.load()

In [22]:
docs[0]

Document(page_content='[TITLE]Genèse 1:1[TITLE] Au commencement, Dieu créa les cieux et la terre.', metadata={'source': '/home/20015587/Documents/IA/bible-search/data/bible_data_v2.fr.json', 'seq_num': 1, 'testament': 'Ancien Testament', 'book': 'Genèse', 'chapter': 1, 'verse': 1, 'title': 'Genèse 1:1'})

In [23]:
#db = Chroma.from_documents(documents=docs, embedding=model, persist_directory="./db/bible_v2.db")
db = Chroma(embedding_function=model, persist_directory="./db/bible_v2.db")


In [31]:
query="Judas a trahi je"
retrieved_docs = db.similarity_search(query,k=2)
retrieved_docs

[Document(page_content='[TITLE]Marc 3:19[TITLE] et Judas Iscariot, celui qui livra Jésus. Ils se rendirent à la maison,', metadata={'book': 'Marc', 'chapter': 3, 'seq_num': 24308, 'source': '/home/20015587/Documents/IA/bible-search/data/bible_data_v2.fr.json', 'testament': 'Nouveau Testament', 'title': 'Marc 3:19', 'verse': 19}),
 Document(page_content="[TITLE]Jean 18:3[TITLE] Judas donc, ayant pris la cohorte, et des huissiers qu'envoyèrent les principaux sacrificateurs et les pharisiens, vint là avec des lanternes, des flambeaux et des armes.", metadata={'book': 'Jean', 'chapter': 18, 'seq_num': 26789, 'source': '/home/20015587/Documents/IA/bible-search/data/bible_data_v2.fr.json', 'testament': 'Nouveau Testament', 'title': 'Jean 18:3', 'verse': 3})]

In [32]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/search')
def search():
    query = request.args.get('q', '')
    if not query:
        return jsonify([])

    # Perform search in Elasticsearch
    response =db.similarity_search(query,k=3)
    response_json = 
   
    return jsonify(response)

def run_app():
    app.run(host='0.0.0.0', port=5000)



In [ ]:
from threading import Thread
flask_thread = Thread(target=run_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.44:5000
Press CTRL+C to quit


127.0.0.1 - - [29/Jun/2024 01:36:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2024 01:36:43] "GET /favicon.ico HTTP/1.1" 404 -
[2024-06-29 01:37:00,522] ERROR in app: Exception on /search [GET]
Traceback (most recent call last):
  File "/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/tmp/ipykernel_2903094/4285742351.py", line 18, in search
  

: 